<a href="https://colab.research.google.com/github/Adityajaiswal03/OLA_caseStudy/blob/main/OLA_caseStudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
!gdown 16g-OJ_4GnG0P8FEFgTa50RMubVSBvIZl

In [ ]:
df=pd.read_csv('/content/ola_driver_scaler.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

## Data Engineering

In [ ]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
df=df.rename(columns={'MMM-YY': 'Reporting_Date'})

In [ ]:
df['Reporting_Date'] = pd.to_datetime(df['Reporting_Date'], format='%m/%d/%y',errors='coerce')
df['Dateofjoining'] = pd.to_datetime(df['Dateofjoining'], format='%d/%m/%y',errors='coerce')
df['LastWorkingDate'] = pd.to_datetime(df['LastWorkingDate'], format='%d/%m/%y',errors='coerce')

In [ ]:
df['Driver_ID'].nunique()

setting the target variable

In [ ]:
target = df.groupby('Driver_ID')['LastWorkingDate'].last().isna().reset_index()
target['LastWorkingDate'] = target['LastWorkingDate'].map({True: 0, False: 1})
target.rename(columns={'LastWorkingDate': 'Target'}, inplace=True)
target.head()


In [ ]:
QR1 =  (df.groupby('Driver_ID').agg({'Quarterly Rating':'first'})['Quarterly Rating']).reset_index()
QR2 = (df.groupby('Driver_ID').agg({'Quarterly Rating':'last'})['Quarterly Rating']).reset_index()

In [ ]:
QR1.shape,QR2.shape

In [ ]:
QR1.isna().sum(),QR2.isna().sum()

In [ ]:
target = target.merge(QR1,on='Driver_ID')
target = target.merge(QR2,on='Driver_ID')

In [ ]:
target['Rating_incr']=np.where(target['Quarterly Rating_x'] < target['Quarterly Rating_y'], 1,0)

In [ ]:
target.head()

In [ ]:
incm1 =  (df.groupby('Driver_ID').agg({'Income':'first'})['Income']).reset_index()
incm2 = (df.groupby('Driver_ID').agg({'Income':'last'})['Income']).reset_index()

In [ ]:
incm1.shape,incm2.shape

In [ ]:
incm1.isna().sum(),incm2.isna().sum()

In [ ]:
target = target.merge(incm1,on='Driver_ID')
target = target.merge(incm2,on='Driver_ID')

In [ ]:
target['Income_incr'] = np.where(target['Income_x'] < target['Income_y'], 1,0)

In [ ]:
target2=target[['Driver_ID','Target','Rating_incr','Income_incr']]

In [ ]:
target2.head()

Aggerigating based on driver ID

In [ ]:
df2=df.copy()

In [ ]:
functions = {'Reporting_Date':'count',
             'Driver_ID':'first',
             'Age':'max',
             'Gender':'last',
            'City':'last',
             'Education_Level':'last',
             'Dateofjoining':'first',
            'LastWorkingDate':'last',
             'Grade':'last',
             'Total Business Value':'sum',
            'Income':'last',
             'Joining Designation':'last',
             'Quarterly Rating':'last'}
df2 = df2.groupby([df2['Driver_ID']]).aggregate(functions)
df2.rename(columns={'Reporting_Date':'Reportings'},inplace=True)

In [ ]:
df2.reset_index(drop=True, inplace=True)
df2 = df2.merge(target2,on='Driver_ID')

In [ ]:
df2.columns = df2.columns.str.strip()

checking for the null values

In [ ]:
df2.isna().sum()

we are getting null values in Last working day as the driver hasn't left the company yet

Outliers

In [ ]:
num_cols=['Reportings','Age','Total Business Value','Income']

In [ ]:
fig = plt.figure(figsize=(10,5))
i=1
for col in num_cols:
  ax = plt.subplot(2,2,i)
  sns.boxplot(x=df2[col],color='yellowgreen')
  plt.title(f'Boxplot of {col}')
  i += 1

plt.tight_layout()
plt.show()

Data is showing high outliers in Total Business value and Income

## EDA

In [ ]:
data=df2.copy()


In [ ]:
data.nunique()

In [ ]:
data.describe()

In [ ]:
cat_cols=['Gender','Education_Level','Grade','Joining Designation','Quarterly Rating','Rating_incr','Income_incr','Target']

In [ ]:
plt.figure(figsize=(10,15))
i=1
for col in cat_cols:
  ax=plt.subplot(4,2,i)
  sns.countplot(x=df2[col])
  plt.title(f'{col}')
  i += 1

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,3))
sns.countplot(x=df2['City'])
plt.title('Distribution of City')

In [ ]:
plt.figure(figsize=(10,3))
sns.countplot(x=df2['LastWorkingDate'].dt.month)
plt.title('Distribution of Month of the Last Working Date')

plt.figure(figsize=(10,3))
sns.countplot(x=df2['LastWorkingDate'].dt.year)
plt.title('Distribution of Year of the Last Working Date')

plt.show()

In [ ]:
plt.figure(figsize=(10,3))
sns.countplot(x=df2['Dateofjoining'].dt.month)
plt.title('Distribution of Month of the Joining Date')

plt.figure(figsize=(10,3))
sns.countplot(x=df2['Dateofjoining'].dt.year)
plt.title('Distribution of Year of the Joining Date')

plt.show()

In [ ]:

grade_wise_value = df2.groupby('Grade')['Total Business Value'].sum()

# Create the plot
plt.bar(grade_wise_value.index, grade_wise_value.values,color='yellowgreen')
plt.xlabel('Grade')
plt.ylabel('Total Business Value')
plt.title('Grade-wise Performance (Total Business Value)')
plt.show()


City with most improvement in Grade rating

In [ ]:
data1=df.copy()

In [ ]:
## with respect to the year of the last reporting Date

data1['Reporting_Date'] = pd.to_datetime(data1['Reporting_Date'])


last_date = data1['Reporting_Date'].max()
one_year_ago = last_date - pd.DateOffset(years=1)


df_past_year = data1[data1['Reporting_Date'] >= one_year_ago]


if df_past_year.empty:
    raise ValueError("No data available for the past year. Please check the date range or the data.")


rating_change = df_past_year.groupby('City').agg(
    start_rating=('Quarterly Rating', 'first'),
    end_rating=('Quarterly Rating', 'last')
).reset_index()


rating_change['rating_improvement'] = rating_change['end_rating'] - rating_change['start_rating']

In [ ]:
if rating_change.empty or rating_change['rating_improvement'].isnull().all():
    raise ValueError("No improvements found. Please check the data.")


most_improved_city = rating_change.loc[rating_change['rating_improvement'].idxmax(), 'City']

print(f'The city with the most improvement in Quarterly Rating over the past year is: {most_improved_city}')


In [ ]:
#Total business value x city

city_tbv = data1.groupby('City')['Total Business Value'].sum().reset_index()


plt.figure(figsize=(10, 6))
sns.barplot(data=city_tbv, x='City', y='Total Business Value')

# Add title and labels
plt.title('Total Business Value by City')
plt.xlabel('City')
plt.ylabel('Total Business Value')
plt.show()

Impact of Each column on churn

In [ ]:
cat1_cols=['Gender','Education_Level','Grade','Joining Designation','Quarterly Rating','Rating_incr','Income_incr']

In [ ]:

plt.figure(figsize=(10,15))
i=1
for col in cat1_cols:
    ax = plt.subplot(4, 2, i)

    data = df2.pivot_table(index=col, columns='Target', aggfunc='size')

    # Convert counts to percentages
    data = data.div(data.sum(axis=1), axis=0).multiply(100).round()
    data.reset_index(inplace=True)

    # Plotting the bars
    plt.bar(data[col], data[1], color='greenyellow',label='Churned')
    plt.bar(data[col], data[0], color='dimgrey', bottom=data[1],label='Not Churned')

    # Labeling and titles
    plt.xlabel(f'{col}')
    plt.ylabel('% Drivers')
    plt.title(f'% Churn by {col}')
    plt.legend(['Churned', 'Not Churned'])

    i += 1

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

In [ ]:

plt.figure(figsize=(10,15))
i=1
for col in cat1_cols:
    ax = plt.subplot(4, 2, i)

    data = df2.pivot_table(index=col, columns='Target', aggfunc='size')

    # Convert counts to percentages
    data = data.div(data.sum(axis=1), axis=0).multiply(100).round()
    data.reset_index(inplace=True)

    # Plotting the bars
    plt.bar(data[col], data[1], color='greenyellow',label='Churned')
    plt.bar(data[col], data[0], color='dimgrey', bottom=data[1],label='Not Churned')

    # Labeling and titles
    plt.xlabel(f'{col}')
    plt.ylabel('% Drivers')
    plt.title(f'% Churn by {col}')
    plt.legend(['Churned', 'Not Churned'])

    i += 1

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

In [ ]:
city = df2.pivot_table(index='City', columns='Target', aggfunc='size')
city = city.div(city.sum(axis=1), axis=0).multiply(100).round()
city.reset_index(inplace=True)
city = city.sort_values(by=1, ascending=False)

plt.figure(figsize=(14,4))
plt.bar(city['City'], city[1], color='greenyellow')
plt.bar(city['City'], city[0], color='dimgrey', bottom=city[1])

# Labeling and titles
plt.xlabel('City')
plt.ylabel('% Drivers')
plt.title(f'% Churn by City')
plt.legend(['Churned', 'Not Churned'])
plt.xticks(rotation=90)
plt.show()

Key notes:

Key Insights on Driver Churn:

* Gender and education level don’t seem to have any impact on churn.
* Grade 1 drivers had the highest churn rate at 80%, followed by Grade 2 at around 70%.
* The highest churn was seen among drivers who started with Designation 1 and 5, with nearly 75% leaving.
* Quarterly Rating appears to influence churn: 80% of those with a QR1 rating left, followed by 40% of QR2 and about 18% of QR3.
* Around 77% of drivers who didn’t see any improvement in their rating ended up leaving.
* Income growth also played a role—70% of those who didn’t get a raise decided to quit.
* Location mattered too—80% of drivers from City C13 left, with C17 and C23 not far behind.



# Correlation

In [ ]:
numerical_df2 = df2.select_dtypes(include=['int64', 'float64'])

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(numerical_df2.corr(), annot=True, fmt=".1f",cmap='Blues')
plt.title('Correlation between Features')
plt.show()

### **Key Takeaways:**  

- **Reportings** show a strong positive correlation with **Total Business Value**.  
- **Quarterly Rating** and **Rating Increase** are closely related, which makes sense.  
- **Grade** has a strong positive correlation with **Income** and **Joining Designations**.  
- Based on these insights, we could consider dropping some features. However, **multicollinearity** isn’t just about individual feature pairs—it can result from multiple variables interacting together.  
- Simply using a correlation threshold to remove features might be misleading, as it doesn’t always capture the full impact on the model.  
- A better approach would be to finalize feature selection using **Feature Importance** analysis.  



## Data prep

In [ ]:
df_prep=df2.drop(columns=['Driver_ID','LastWorkingDate'],axis=1)

In [ ]:
df_prep['Month']=df_prep['Dateofjoining'].dt.month
df_prep['Year']=df_prep['Dateofjoining'].dt.year

In [ ]:
df_prep.drop('Dateofjoining',axis=1,inplace=True)

In [ ]:
df_encoded = pd.get_dummies(df_prep,'City', drop_first=True)*1

In [ ]:
df_encoded.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df_encoded.drop(['Target'], axis=1)
y = df_encoded['Target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

In [ ]:
class_counts = y_train.value_counts()


plt.figure(figsize=(6, 4))
class_counts.plot(kind='bar', color=['greenyellow', 'dimgrey'])
plt.xlabel('Class Label')
plt.ylabel('Number of Samples')
plt.title('Class Distribution')
plt.xticks(rotation=0)
plt.show()

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

In [ ]:
class_counts = y_train_res.value_counts()


plt.figure(figsize=(6, 4))
class_counts.plot(kind='bar', color=['greenyellow', 'dimgrey'])
plt.xlabel('Class Label')
plt.ylabel('Number of Samples')
plt.title('Class Distribution')
plt.xticks(rotation=0)
plt.show()

# Bagging(Random Forest Classifier)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time

In [ ]:
params = {"max_depth":[7,10,15],
             "n_estimators":[100,200,300,400],
             "max_features":[4,7,10],
             "ccp_alpha":[0.0005,0.00075,0.001]}

In [ ]:
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=params, cv=5, n_jobs=-1, verbose=2)

# Measure the time taken to fit the model
start_time = time.time()
grid_search.fit(X_train_res, y_train_res)
end_time = time.time()

print("Best parameters found by GridSearchCV:", grid_search.best_params_)
print(f"Total training time: {end_time - start_time:.2f} seconds")

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay


In [ ]:

# Retrieve the best model (estimator)
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_train_pred = best_model.predict(X_train_res)
y_test_pred = best_model.predict(X_test_scaled)

# Evaluate the model
# Accuracy
train_accuracy = accuracy_score(y_train_res, y_train_pred)
print(f"Training Accuracy: {train_accuracy:.2f}")

test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.2f}")


In [ ]:
grid_search.best_score_


### **Observations:**  

- **Training Accuracy: 0.98** – The Random Forest model achieved **98% accuracy** on the training data, indicating that it fits the training set very well.  

- **Test Accuracy: 0.89** – When tested on unseen data, the model reached **89% accuracy**, showing that it generalizes well. The slight drop from training accuracy is normal and expected.  

- **Best Model Score: 0.915** – This represents the highest score obtained during cross-validation, likely using **GridSearchCV** for hyperparameter tuning. A score of **0.915** suggests that the model performed well across different validation sets.  

Overall, the model demonstrates **strong performance**, balancing accuracy on both training and test datasets.  
